In [ ]:
df = pd.read_csv('../Testing/stage_2_train_labels.csv')
#print(df.size)
my = df.iloc[0]
myid = my['patientId']
print("the id is ")
print(myid)
myfile = '../Testing/stage_2_train_images/%s.dcm' % myid 
mypix = pydicom.read_file(myfile)
pixels = mypix.pixel_array
print(pixels.shape)
torch_variable = torch.from_numpy(pixels)

In [ ]:
patientId = df['patientId'][0]
dcm_file = '../Testing/stage_2_train_images/%s.dcm' % patientId
dcm_data = pydicom.read_file(dcm_file)
print(dcm_data)
print(type(dcm_data))

In [ ]:
im = dcm_data.pixel_array
image = im[..., np.newaxis]
image = image.transpose((2, 0, 1))
#print(image.shape)
#print(image.shape[2])
print(image[:,:,1].shape)

In [ ]:
pylab.imshow(im, cmap=pylab.cm.gist_gray)
pylab.axis('off')
import matplotlib.pyplot as plt
image = plt.imshow(im)
plt.show()

In [ ]:
def parse_data(df):
    """
    Method to read a CSV file (Pandas dataframe) and parse the 
    data into the following nested dictionary:

      parsed = {
        
        'patientId-00': {
            'dicom': path/to/dicom/file,
            'label': either 0 or 1 for normal or pnuemonia, 
            'boxes': list of box(es)
        },
        'patientId-01': {
            'dicom': path/to/dicom/file,
            'label': either 0 or 1 for normal or pnuemonia, 
            'boxes': list of box(es)
        }, ...

      }

    """
    # --- Define lambda to extract coords in list [y, x, height, width]
    extract_box = lambda row: [row['y'], row['x'], row['height'], row['width']]

    parsed = {}
    for n, row in df.iterrows():
        # --- Initialize patient entry into parsed 
        pid = row['patientId']
        if pid not in parsed:
            parsed[pid] = {
                'dicom': '../Testing/stage_2_train_images/%s.dcm' % pid,
                'label': row['Target'],
                'boxes': []}

        # --- Add box if opacity is present
        if parsed[pid]['label'] == 1:
            parsed[pid]['boxes'].append(extract_box(row))

    return parsed

In [ ]:
parsed = parse_data(df)
print(parsed['00c0b293-48e7-4e16-ac76-9269ba535a62'])

In [ ]:
def draw(data):
    """
    Method to draw single patient with bounding box(es) if present 

    """
    # --- Open DICOM file
    d = pydicom.read_file(data['dicom'])
    im = d.pixel_array

    # --- Convert from single-channel grayscale to 3-channel RGB
    im = np.stack([im] * 3, axis=2)

    # --- Add boxes with random color if present
    for box in data['boxes']:
        rgb = np.floor(np.random.rand(3) * 256).astype('int')
        im = overlay_box(im=im, box=box, rgb=rgb, stroke=6)

    pylab.imshow(im, cmap=pylab.cm.gist_gray)
    pylab.axis('off')

def overlay_box(im, box, rgb, stroke=1):
    """
    Method to overlay single box on image

    """
    # --- Convert coordinates to integers
    box = [int(b) for b in box]
    
    # --- Extract coordinates
    y1, x1, height, width = box
    y2 = y1 + height
    x2 = x1 + width

    im[y1:y1 + stroke, x1:x2] = rgb
    im[y2:y2 + stroke, x1:x2] = rgb
    im[y1:y2, x1:x1 + stroke] = rgb
    im[y1:y2, x2:x2 + stroke] = rgb

    return im

In [ ]:
##HOW TO LIST THE NAMES OF FILES IN A DIRECTORY
import os 
a = os.listdir("../Testing/stage_2_test_images/")
print(len(a))
b = a[145]
print(type(b))
print(b)
c = b[:-4]
print(type(c))
print(c)
myFileDir = "../Testing/stage_2_test_images/" + b
print(myFileDir)
dcm_data = pydicom.read_file(myFileDir)
image = dcm_data.pixel_array
print(image)

In [8]:
import pandas as pd
training = pd.read_csv("/Users/abdulsalamyazid/Desktop/Projects/Dataset/train_label.csv")
print(len(training))

30227
